In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import current_timestamp, lit
from pyspark.sql.types import StructType, StructField, TimestampType, StringType

CATALOGO_ORIGEM = "v_credit"
SCHEMA_ORIGEM = "postgres_public"
CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "bronze"

In [0]:
tabelas_postgres = {
    "base_atendentes": "base_atendentes",
    "base_motivos": "base_motivos",
    "canais": "canais",
    "chamados": "chamados",
    "chamados_hora": "chamados_hora",
    "clientes": "clientes",
    "custos": "custos",
    "pesquisa_satisfacao": "pesquisa_satisfacao"
}

In [0]:
for tabela_origem, tabela_destino in tabelas_postgres.items():
    
    full_source_table = f"{CATALOGO_ORIGEM}.{SCHEMA_ORIGEM}.{tabela_origem}"
    full_target_table = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{tabela_destino}"
    
    try:
        print(f"Extraindo schema de: '{full_source_table}'")
        
        source_schema = spark.table(full_source_table).schema
        
        new_schema = StructType(
            source_schema.fields + [
                StructField("ingestion_timestamp", TimestampType(), True),
                StructField("origem", StringType(), True)
            ]
        )
        

        if spark.catalog.tableExists(full_target_table):
            drop_query = f"DROP TABLE {full_target_table}"
            spark.sql(drop_query)
        
        DeltaTable.create(spark) \
            .tableName(full_target_table) \
            .addColumns(new_schema) \
            .property("delta.enableChangeDataFeed", "true") \
            .property("delta.autoOptimize.optimizeWrite", "true") \
            .property("delta.autoOptimize.autoCompact", "true") \
            .execute()
        
        print(f"✅ Tabela '{full_target_table}' criada com sucesso!\n")
        
    except Exception as e:
        print(f"❌ Erro ao criar tabela '{full_target_table}': {str(e)}\n")